In [1]:
# lr:0.001
# anneal_rate:0.9
# batch_size:32
# clip_norm:50
# num_epochs:5
# alpha:250
# beta:0
# max_beta:1
# step_beta:0.002
# anneal_iter:40000
# kl_anneal_iter:2000
# print_iter:100
# save_iter:5000
# num_workers:4

In [2]:
!pip install -q torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cu116.html
!pip install -q dive-into-graphs

In [3]:
!pip install -q toolz
!pip install -q wandb

In [34]:
%load_ext autoreload
%autoreload 2
import os
import json
import argparse
import pickle 

import numpy as np
import pandas as pd
import torch

from molecule_optimizer.externals.fast_jtnn.datautils import SemiMolTreeFolder, SemiMolTreeFolderTest
from molecule_optimizer.runner.semi_jtvae import SemiJTVAEGeneratorPredictor
from torch_geometric.data import DenseDataLoader

import rdkit

lg = rdkit.RDLogger.logger() 
lg.setLevel(rdkit.RDLogger.CRITICAL)

import warnings
warnings.filterwarnings("ignore")

import math

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
conf = json.load(open("training/configs/rand_gen_zinc250k_config_dict.json"))

In [6]:
csv = pd.read_csv("ZINC_310k.csv")

In [7]:
smiles = csv['SMILES']

In [8]:
if 'runner.xml' not in os.listdir("."):
    runner = SemiJTVAEGeneratorPredictor(smiles)
    with open('runner.xml', 'wb') as f:
        pickle.dump(runner, f)

In [9]:
with open('runner.xml', 'rb') as f:
    runner = pickle.load(f)

In [10]:
labels = torch.tensor(csv['QED']).float()

In [11]:
runner.get_model(
    "rand_gen",
    {
        "hidden_size": conf["model"]["hidden_size"],
        "latent_size": conf["model"]["latent_size"],
        "depthT": conf["model"]["depthT"],
        "depthG": conf["model"]["depthG"],
        "label_size": 1,
        "label_mean": float(torch.mean(labels)),
        "label_var": float(torch.var(labels)),
    },
)

In [12]:
labels = runner.get_processed_labels(labels)
preprocessed = runner.processed_smiles

In [13]:
N_TEST = 10000
VAL_FRAC = 0.05

In [14]:
perm_id=np.random.permutation(len(labels))
X_train = preprocessed[perm_id[N_TEST:]]
L_train = torch.tensor(labels.numpy()[perm_id[N_TEST:]])


X_test = preprocessed[perm_id[:N_TEST]]
L_test = torch.tensor(labels.numpy()[perm_id[:N_TEST]])

In [20]:
val_cut = math.floor(len(X_train) * VAL_FRAC)

14109

In [21]:
X_Val = X_train[:val_cut]
L_Val = L_train[:val_cut]

X_train = X_train[val_cut :]
L_train = L_train[val_cut :]

In [25]:
train_loader = SemiMolTreeFolder(
    X_train,
    L_train,
    runner.vocab,
    conf["batch_size"],
    label_pct=0.05,
    num_workers=conf["num_workers"],
)

In [26]:
test_loader = SemiMolTreeFolderTest(
    X_test,
    L_test,
    runner.vocab,
    conf["batch_size"],
    num_workers=conf["num_workers"],
)

In [27]:
val_loader = SemiMolTreeFolderTest(
    X_Val,
    L_Val,
    runner.vocab,
    conf["batch_size"],
    num_workers=conf["num_workers"],
)

In [ ]:
print("Training model...")
runner.train_gen_pred(
    loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    load_epoch=0,
    lr=conf["lr"],
    anneal_rate=conf["anneal_rate"],
    clip_norm=conf["clip_norm"],
    num_epochs=conf["num_epochs"],
    alpha=conf["alpha"],
    beta=conf["beta"],
    max_beta=conf["max_beta"],
    step_beta=conf["step_beta"],
    anneal_iter=conf["anneal_iter"],
    kl_anneal_iter=conf["kl_anneal_iter"],
    print_iter=100,
    save_iter=conf["save_iter"],
)

Training model...
Model #Params: 5207K
[Train][100] Alpha: 250.000, Beta: 0.000, Loss: 320.92, KL: 70.07, MAE: 0.00109, Word Loss: 95.06, Topo Loss: 25.57, Assm Loss: 8.51, Pred Loss: 0.77, Word: 0.26, Topo: 0.80, Assm: 0.57, PNorm: 104.01, GNorm: 50.00
[Validation][100] Alpha: 250.000, Beta: 0.000, Loss: 166.67, KL: 40.38, MAE: 0.00088, Word Loss: 33.65, Topo Loss: 8.85, Assm Loss: 4.00, Pred Loss: 0.48, Word: 0.39, Topo: 0.88, Assm: 0.60
